In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from util import OHLC_COLUMNS, add_features, download_data
from util import preprocess

df = download_data()

dataset = df.loc[:, OHLC_COLUMNS].astype(np.float64)
dataset = pd.DataFrame(dataset.to_numpy(), columns=OHLC_COLUMNS)

dataset = add_features(dataset)
preprocessed_dataset, scaler = preprocess(dataset)

['ETHBTC', 'LTCBTC', 'BNBBTC', 'NEOBTC', 'QTUMETH', 'EOSETH', 'SNTETH', 'BNTETH', 'BCCBTC', 'GASBTC', 'BNBETH', 'BTCUSDT', 'ETHUSDT', 'HSRBTC', 'OAXETH', 'DNTETH', 'MCOETH', 'ICNETH', 'MCOBTC', 'WTCBTC', 'WTCETH', 'LRCBTC', 'LRCETH', 'QTUMBTC', 'YOYOBTC', 'OMGBTC', 'OMGETH', 'ZRXBTC', 'ZRXETH', 'STRATBTC', 'STRATETH', 'SNGLSBTC', 'SNGLSETH', 'BQXBTC', 'BQXETH', 'KNCBTC', 'KNCETH', 'FUNBTC', 'FUNETH', 'SNMBTC', 'SNMETH', 'NEOETH', 'IOTABTC', 'IOTAETH', 'LINKBTC', 'LINKETH', 'XVGBTC', 'XVGETH', 'SALTBTC', 'SALTETH', 'MDABTC', 'MDAETH', 'MTLBTC', 'MTLETH', 'SUBBTC', 'SUBETH', 'EOSBTC', 'SNTBTC', 'ETCETH', 'ETCBTC', 'MTHBTC', 'MTHETH', 'ENGBTC', 'ENGETH', 'DNTBTC', 'ZECBTC', 'ZECETH', 'BNTBTC', 'ASTBTC', 'ASTETH', 'DASHBTC', 'DASHETH', 'OAXBTC', 'ICNBTC', 'BTGBTC', 'BTGETH', 'EVXBTC', 'EVXETH', 'REQBTC', 'REQETH', 'VIBBTC', 'VIBETH', 'HSRETH', 'TRXBTC', 'TRXETH', 'POWRBTC', 'POWRETH', 'ARKBTC', 'ARKETH', 'YOYOETH', 'XRPBTC', 'XRPETH', 'MODBTC', 'MODETH', 'ENJBTC', 'ENJETH', 'STORJBTC', 'ST

Tickers:   0%|          | 0/1 [00:00<?, ?it/s]

monthly files to download: 0files [00:00, ?files/s]

daily files to download: 0files [00:00, ?files/s]

Tried to dump data for 1 tickers:
---> For NEARUSDT new data saved for: 0 months 0 days


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/util.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


In [3]:
from util import test_preprocess_invert_preprocess, test_orig_val

test_preprocess_invert_preprocess(dataset)
test_orig_val(dataset)

Mean Absolute Error for Open: 1.107702642891245
Mean Absolute Error for High: 1.3524301038831599
Mean Absolute Error for Low: 0.7988906677050388
Mean Absolute Error for Close: 1.1182623099180922
Mean Absolute Error for SMA_256: 7.725245422375353e-18
Mean Absolute Error for SMA_512: 7.540261310804373e-18
Mean Absolute Error for SMA_1024: 6.0620489304659e-18
Mean Absolute Error for MACD_diff: 7.061264968818638e-18
Mean Absolute Error for RSI: 4.1411930450655985e-17
Mean Absolute Error for CCI: 2.7459341034366415e-15
Mean Absolute Error for ADX: 8.666418617795355e-16
Mean Absolute Error for stoch: 3.4714785398373443e-16
Average MAE: 0.36477381036646167


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/model-weights/")
# except:
#   pass
# my_model.save(f"./drive/MyDrive/colab/model-weights/model-weights-{time.time()}.pt")

In [5]:
%load_ext tensorboard
%tensorboard --logdir tensorboard

Launching TensorBoard...

In [6]:
from model import TransformerExtractor, LSTMExtractor
from env import CustomEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(dataset, scaler)
check_env(env)
del env

env = make_vec_env(lambda: CustomEnv(dataset, scaler), n_envs=10, seed=42, vec_env_cls=SubprocVecEnv)
# env = make_vec_env(lambda: CustomEnv(dataset, scaler), n_envs=3, seed=42, vec_env_cls=SubprocVecEnv)

policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
                     features_extractor_class=LSTMExtractor,
                     features_extractor_kwargs=dict(lstm_hidden_size=128, lstm_layers=2),
                     net_arch=[256, 256, 256])
# policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
#                      features_extractor_class=TransformerExtractor,
#                      features_extractor_kwargs=dict(transformer_hidden_size=256, transformer_heads=4,
#                                                     transformer_layers=2),
#                      net_arch=[256, 256, 256])
# {'gamma': 0.8, 'ent_coef': 0.02, 'gae_lambda': 0.92}
rl_model = PPO("MultiInputPolicy", env,
               verbose=1,
               tensorboard_log="./tensorboard/",
               ent_coef=0.02,
               gae_lambda=0.92,
               gamma=0.8,
               policy_kwargs=policy_kvargs)
# rl_model.set_parameters("rl-model-best-model/best_model/best_model.zip")
rl_model.policy

obs length = {'prices_sequence': (256, 12), 'other': (2,)}


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation prices_sequence has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device


MultiInputActorCriticPolicy(
  (features_extractor): LSTMExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(12, 128, num_layers=2, batch_first=True)
    )
  )
  (pi_features_extractor): LSTMExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(12, 128, num_layers=2, batch_first=True)
    )
  )
  (vf_features_extractor): LSTMExtractor(
    (extractors): ModuleDict(
      (other): Flatten(start_dim=1, end_dim=-1)
      (prices_sequence): LSTM(12, 128, num_layers=2, batch_first=True)
    )
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=130, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=256, bias=True)
      (5): LeakyReLU(negative_sl

In [7]:
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="rl-model-checkpoints/",
    name_prefix="rl_model",
    verbose=1,
    save_vecnormalize=True,
    save_replay_buffer=True
)
eval_env = make_vec_env(lambda: CustomEnv(dataset, scaler, episode_length=2048), n_envs=1, seed=42,
                        vec_env_cls=SubprocVecEnv)
eval_callback = EvalCallback(eval_env, best_model_save_path="rl-model-best-model/best_model",
                             log_path="rl-model-best-model/results", eval_freq=50_000, verbose=1,
                             n_eval_episodes=5)

rl_model.learn(total_timesteps=1_000_000, callback=[checkpoint_callback, eval_callback])

Logging to ./tensorboard/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | -1.35    |
| time/              |          |
|    fps             | 237      |
|    iterations      | 1        |
|    time_elapsed    | 86       |
|    total_timesteps | 20480    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+03    |
|    ep_rew_mean          | -1.29       |
| time/                   |             |
|    fps                  | 126         |
|    iterations           | 2           |
|    time_elapsed         | 323         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.010155757 |
|    clip_fraction        | 0.05        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.453      |

In [8]:
evaluations = np.load("rl-model-best-model/results/evaluations.npz")
evaluations_df = pd.DataFrame()
evaluations_df["timesteps"] = evaluations["timesteps"]
evaluations_df["results"] = evaluations["results"].mean(axis=1)
evaluations_df

,timesteps,results
0,500000,-2.026398
1,1000000,-3.814865


In [9]:
from util import SKIP_STEPS, calculate_observation
# backtesting

from backtesting import Backtest, Strategy, set_bokeh_output


class NeuralNetStrat(Strategy):
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.buy_price = None
        self.expected_gain = None

    def init(self):
        pass

    def next(self):
        if len(self.data) > SKIP_STEPS:
            df = self.data.df.iloc[-SKIP_STEPS:].copy()
            df.drop(columns=["Volume"], inplace=True)
            observation, curr_close, _ = calculate_observation(df, scaler, self.buy_price)

            action, _ = rl_model.predict(observation, deterministic=True)
            if self.buy_price is None:
                if action == 1:
                    self.buy()
                    print(f"buy")
                    self.buy_price = curr_close
            else:
                if action == 2:
                    commission = 0.001
                    sell_fee = curr_close * (1 - commission)
                    buy_fee = self.buy_price * (1 + commission)
                    gain_from_trade_fee = (sell_fee - buy_fee) / buy_fee
                    print(f"gain {gain_from_trade_fee}")
                    self.sell()
                    self.buy_price = None


backtest_dataset = dataset[-20000:]
backtest_dataset["Volume"] = df[1023:]["Volume"]
bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,
              exclusive_orders=True)
stats = bt.run()
stats

/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ... 
 
 Warning: Requesting CDN BokehJS version '3.3.0rc2' from local development version '3.3.0rc2+2.g273f1258'. This configuration is unsupported and may not work!

/tmp/ipykernel_10337/2034666175.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest_dataset["Volume"] = df[1023:]["Volume"]
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/tmp/ipykernel_10337/2034666175.py:41: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,


buy
gain -0.0019980019980018315
buy
gain -0.0009969969849608449
buy
gain -0.0029980040000079783
buy
gain -0.0019980019980018805
buy
gain -0.0029980040000079783
buy
gain 0.0009990009990011933
buy
gain -0.0029960039960038594
buy
gain -0.0029960039960038594
buy
gain -0.0029970029970029137
buy
gain -0.0019980019980018315
buy
gain -0.004998008004019942
buy
gain -0.003003039256815596
buy
gain 3.6660587119452446e-05
buy
gain 0.0030731463658294077
buy
gain -0.0019980019980019715
buy
gain -0.0019980019980019134
buy
gain -0.0030163713837181422
buy
gain -0.005053110155150848
buy
gain 0.0010664872486777575
buy
gain -0.004026461343534428
buy
gain -0.005046887938944846
buy
gain 3.6660587119452446e-05
buy
gain -0.0019980019980019633
buy
gain -0.0019980019980019633
buy
gain -0.00097963261228555
buy
gain -0.0019980019980019633
buy
gain -0.00097963261228555
buy
gain -0.001998001998001846
buy
gain -0.001998001998001846
buy
gain -0.004034740769434552
buy
gain -0.000978592398408355
buy
gain 3.8736773430746

Start                               1580996.0
End                                 1600995.0
Duration                              19999.0
Exposure Time [%]                      93.595
Equity Final [$]                   196.581992
Equity Peak [$]                   10000.03001
Return [%]                          -98.03418
Buy & Hold Return [%]               32.405567
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -98.04147
Avg. Drawdown [%]                  -32.747156
Max. Drawdown Duration                18715.0
Avg. Drawdown Duration            6239.666667
# Trades                               4108.0
Win Rate [%]                        21.445959
Best Trade [%]                       1.931941
Worst Trade [%]                     -1.307095
Avg. Trade [%]                    

In [10]:
set_bokeh_output(notebook=False)
bt.plot()

INFO:bokeh.io.state:Session output file 'NeuralNetStrat.html' already exists, will be overwritten.
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1265', ...)